In [1]:
from keras.models import Sequential  
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout
from keras import layers, models, optimizers, losses
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2 
import os 
import glob

In [2]:
# CARREGAMENTO DOS DATASETS

''' 
    Carregando a pasta de imagens (Dataset dos ovos)
    * A função glob encontra todos os nomes de caminhos que correspondem a um padrão 
      especificado de acordo com as regras usadas pelo shell Unix

    * Para cada um dos arquivos encontrados é feita a leitura
      do arquivo .jpg (imagem), as imagens lidas são armazenadas
      na lista 'data'
'''
img_dir = "C:/Users/marci/Desktop/Projeto mestrado/Images Dataset/Batch 06.04 Images"
data_path = os.path.join(img_dir,'*g') 

folder = glob.glob(data_path) 
data = [] 
for files in folder: 
    img = cv2.imread(files) 
    data.append(img) 

'''   
    Carregando e tratando o arquivo CSV
'''
eggClass = pd.read_csv("RealData.csv")
cList = eggClass.to_numpy()

print(len(data))
print(cList.shape)

5100
(5100, 1)


In [4]:
subset = []
values = []
for i in range(len(cList)):
    if cList[i] == 0:
        subset.append(i)
    if cList[i] == 1:
        values.append(i)

randomIndex = np.random.choice(values, 1373, replace=False)

for j in range(len(randomIndex)):
    subset.append(randomIndex[j])

print(subset)
print(len(subset))


[1, 5, 12, 14, 18, 20, 33, 43, 45, 57, 60, 61, 62, 68, 74, 75, 76, 79, 87, 88, 90, 92, 97, 103, 109, 112, 113, 114, 116, 123, 126, 136, 138, 141, 144, 145, 148, 149, 154, 156, 157, 165, 172, 185, 186, 192, 194, 199, 201, 206, 210, 216, 220, 222, 223, 225, 230, 231, 232, 236, 240, 241, 244, 247, 248, 253, 255, 257, 258, 266, 270, 271, 276, 283, 285, 289, 297, 298, 303, 306, 316, 331, 343, 349, 353, 355, 356, 360, 365, 366, 368, 369, 371, 372, 379, 390, 391, 397, 405, 410, 414, 416, 418, 420, 421, 438, 439, 442, 444, 445, 446, 447, 448, 449, 451, 454, 456, 457, 459, 467, 469, 473, 474, 479, 482, 485, 486, 488, 489, 493, 500, 510, 514, 515, 522, 523, 524, 527, 529, 534, 537, 538, 548, 554, 565, 568, 571, 572, 581, 583, 588, 590, 597, 598, 603, 608, 610, 612, 616, 617, 618, 620, 621, 624, 625, 626, 628, 635, 638, 641, 643, 645, 646, 647, 653, 654, 656, 662, 668, 672, 678, 682, 689, 692, 695, 697, 700, 706, 707, 711, 715, 720, 721, 739, 741, 744, 750, 752, 754, 757, 759, 760, 761, 762, 774,

In [5]:
ct1 = 0
ct2 = 0
for z in range(len(subset)):
    if cList[subset[z]] == 0:
        ct1 += 1
    if cList[subset[z]] == 1:
        ct2 += 1
    
print(ct1)
print(ct2)

1373
1373


In [6]:
# Dataset Completo
# Comentar seção caso não seja necessário

#ind = np.arange(len(data))
train, test = train_test_split(subset, test_size=0.2, random_state=42)

#
dataTrain = []
dataTest = []
classTrain = []
classTest = []

#
for i in range(len(train)):

    dataTrain.append(data[train[i]])
    classTrain.append(cList[train[i]])

for j in range(len(test)):

    dataTest.append(data[test[j]])
    classTest.append(cList[test[j]])

#
dataTrain = np.array(dataTrain)
dataTest = np.array(dataTest)
classTrain = np.array(classTrain)
classTest = np.array(classTest)

print('img train', len(dataTrain))
print('img test', len(dataTest))
print('class train', len(classTrain))
print('class test', len(classTest))


img train 2196
img test 550
class train 2196
class test 550


In [7]:
del data
del cList
del subset
del values
del randomIndex

In [8]:
'''
    Alterando o tipo dos dados para float32 a fim de aplicar 
    a normalização futuramente
'''

dataTrain = dataTrain.astype('float32')
dataTest = dataTest.astype('float32')

'''
    Realizando a normalização (min/max normalization) a fim de que os valores dos pixels estejam
    entre 0 e 1, tornando o processamento mais eficiente
    obs: 255 é o valor máximo do pixel
'''

dataTrain /= 255
dataTest /= 255

In [3]:
# SHALLOW NETWORK

classifier = Sequential()

classifier.add(Conv2D(32, (3,3), input_shape = (512, 512, 3), activation = 'relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2,2)))

classifier.add(Conv2D(32, (3,3), activation = 'relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2,2)))

classifier.add(Flatten())

classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(0.2))
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(0.2))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

classifier.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

C:\Users\marci\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [9]:
# ALEXNET

model = Sequential()

model.add(Conv2D(96, (11, 11), strides=4, activation='relu', input_shape = (512, 512, 3), padding='valid'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))
    
model.add(Conv2D(256, (5, 5), strides=1, activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))
    
model.add(Conv2D(384, (3, 3), strides=1, activation='relu', padding='same'))
    
model.add(Conv2D(384, (3, 3), strides=1, activation='relu', padding='same'))
    
model.add(Conv2D(256, (3, 3), strides=1, activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))
    
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units = 1, activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [10]:
# MOBILE NET PRÉ-TREINADO

base_model = tf.keras.applications.MobileNetV2(input_shape=(512, 512, 3),
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False  # Congelar as camadas do modelo base

model2 = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model2.compile(optimizer=optimizers.Adam(learning_rate=0.001),
              loss=losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

C:\Users\marci\AppData\Local\Temp\ipykernel_7028\2331232953.py:3: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNetV2(input_shape=(512, 512, 3),


In [10]:
# TREINO ALEXNET

board = TensorBoard(log_dir='./logs3AlexSubset50x50')
model.fit(dataTrain, classTrain, batch_size = 32, epochs = 10, validation_data = (dataTest, classTest), callbacks=[board])

Epoch 1/10


69/69 [==============================] - 362s 5s/step - loss: 1.1878 - accuracy: 0.4886 - val_loss: 0.6933 - val_accuracy: 0.5055
Epoch 2/10
69/69 [==============================] - 349s 5s/step - loss: 0.6941 - accuracy: 0.4959 - val_loss: 0.6931 - val_accuracy: 0.5055
Epoch 3/10
69/69 [==============================] - 347s 5s/step - loss: 0.6935 - accuracy: 0.4959 - val_loss: 0.6931 - val_accuracy: 0.5055
Epoch 4/10
69/69 [==============================] - 345s 5s/step - loss: 0.6937 - accuracy: 0.4863 - val_loss: 0.6931 - val_accuracy: 0.5055
Epoch 5/10
69/69 [==============================] - 348s 5s/step - loss: 0.6934 - accuracy: 0.5000 - val_loss: 0.6938 - val_accuracy: 0.4945
Epoch 6/10
69/69 [==============================] - 344s 5s/step - loss: 0.6934 - accuracy: 0.4982 - val_loss: 0.6931 - val_accuracy: 0.5055
Epoch 7/10
69/69 [==============================] - 345s 5s/step - loss: 0.6940 - accuracy: 0.4927 - val_loss: 0.6931 - val_accuracy: 0.5055
Epoch 8/10


In [ ]:
board2 = TensorBoard(log_dir='./logsShallow')
classifier.fit(dataTrain, classTrain, batch_size = 32, epochs = 10, validation_data = (dataTest, classTest), callbacks=[board2])

In [ ]:
board3 = TensorBoard(log_dir='./logsMobile')
model2.fit(dataTrain, classTrain, batch_size = 32, epochs = 10, validation_data = (dataTest, classTest), callbacks=[board3])


In [11]:
model.save_weights('AlexNet3Subset50x50.weights.h5')
#model2.save_weights('mobileNet10ep.weights.h5')
#classifier.save_weights('shallownet.weights.h5')

In [5]:
# Carregando pesos

model.load_weights('AlexNet10ep.weights.h5')
classifier.load_weights('shallownet.weights.h5')

C:\Users\marci\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 34 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))
C:\Users\marci\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 30 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [67]:
npData = np.array(data)
alexPredict = model.predict(npData)
#shallowPredict = classifier.predict(npData)


160/160 [==============================] - 164s 991ms/step


In [68]:
npAlex = np.array(alexPredict)
#npShallow = np.array(shallowPredict)

print(npAlex)
print('-----------------')
#print(npShallow)

[[0.8320737]
 [0.8259779]
 [0.8212664]
 ...
 [0.8033839]
 [0.8216767]
 [0.7940623]]
-----------------


In [69]:
AlexClass = (npAlex >= 0.5).astype(int)
#ShallowClass = (npShallow >= 0.5).astype(int)

In [71]:
AlexClass = np.ravel(AlexClass)
#ShallowClass = np.ravel(ShallowClass)
print(AlexClass)
#print(ShallowClass)

[1 1 1 ... 1 1 1]


In [60]:
count = 0
count2 = 0
for i in range(len(AlexClass)):
    if AlexClass[i] == 1:
        count += 1
    
    if ShallowClass[i] == 1:
        count2 += 1

print('alex',count)
print('shallow',count2)

alex 5100
shallow 3559
